### QG_data_V1

Question generation model 깃 주소
https://github.com/codertimo/KorQuAD-Question-Generation

Question generation source - 토론게시판 문하겸 캠퍼님
https://stages.ai/competitions/77/discussion/talk/post/769


깃에서 학습된 QG 모델을 다운로드 해주시고 load_state_dict() 에서 위치를 설정해주세요

In [27]:
import pandas as pd
import json
from datasets import load_dataset, load_from_disk
import datasets
from time import sleep
from pprint import pprint

### 학습데이터 불러오기

In [5]:
wiki_df = pd.read_json("../../data/wikipedia_documents.json", orient='index')
wiki_df.head()

,text,corpus_source,url,domain,title,author,html,document_id
0,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",위키피디아,TODO,NaN,나라 목록,NaN,NaN,0
1,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,위키피디아,TODO,NaN,나라 목록,NaN,NaN,1
2,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,위키피디아,TODO,NaN,백남준,NaN,NaN,2
3,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",위키피디아,TODO,NaN,아오조라 문고,NaN,NaN,3
4,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,위키피디아,TODO,NaN,아오조라 문고,NaN,NaN,4


In [18]:
texts = []
titles = []

for i in range(len(wiki_df)):
    wiki_context = wiki_df['text'][i]
    wiki_title = wiki_df['title'][i]

    if wiki_title in wiki_context:
        texts.append(wiki_context)
        titles.append(wiki_title)

wiki_qa_df = pd.DataFrame(data={'text':texts,'title':titles})
wiki_qa_df.head()

,text,title
0,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",나라 목록
1,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,백남준
2,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",아오조라 문고
3,아오조라 문고는 자원봉사로 운영되며 열람 역시 무료이다. 서비스 개시 초반에는 보...,아오조라 문고
4,"텍스트 파일을 아오조라 문고에 수록할 때, 텍스트 파일이 갖추어야 할 서식을 '아오...",아오조라 문고


In [19]:
wiki_qa_df.to_csv('/opt/ml/data/wiki_text_title.csv')

In [9]:
!git clone https://github.com/codertimo/KorQuAD-Question-Generation

Cloning into 'KorQuAD-Question-Generation'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 125 (delta 54), reused 105 (delta 39), pack-reused 0
Receiving objects: 100% (125/125), 926.44 KiB | 2.59 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [16]:
sys.path.append("KorQuAD-Question-Generation")
from korquad_qg.dataset import QAExample

def load_wiki_dataset(dataset_path):
    wiki_data_frame = pd.read_csv(dataset_path)
    examples = []
    for idx in range(len(wiki_data_frame)):
        text = wiki_data_frame["text"][idx]
        title = wiki_data_frame["title"][idx]

        example = QAExample(text, title)
        examples.append(example)
    return examples

### 질문 생성

In [ ]:
import sys
sys.path.append("..")
import random
from argparse import ArgumentParser

import torch
from tokenizers import SentencePieceBPETokenizer
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import GPT2LMHeadModel

sys.path.append("KorQuAD-Question-Generation")
from korquad_qg.config import QGConfig
from korquad_qg.dataset import (MAX_QUESTION_SPACE, MIN_QUESTION_SPACE, QGDecodingDataset)

import pandas as pd

model = GPT2LMHeadModel.from_pretrained("taeminlee/kogpt2")

# 여기서 다운로드한 모델 경로 설정해주세요
model.load_state_dict(torch.load('./KorQuAD-Question-Generation/QG_kogpt2.pth', map_location="cpu"))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

tokenizer = SentencePieceBPETokenizer.from_file(
    vocab_filename="./KorQuAD-Question-Generation/tokenizer/vocab.json", merges_filename="./KorQuAD-Question-Generation/tokenizer/merges.txt", add_prefix_space=False
)

examples = load_wiki_dataset('/opt/ml/data/wiki_text_title.csv')
random.shuffle(examples)
dataset = QGDecodingDataset(examples, tokenizer, 512)
dataloader = DataLoader(dataset, batch_size=1)

model = model.to(device)
model.eval()

generated_results = []

for i, batch in tqdm(enumerate(dataloader), desc="generate", total=len(dataloader)):
    input_ids, attention_mask = (v.to(device) for v in batch)
    origin_seq_len = input_ids.size(-1)

    decoded_sequences = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=origin_seq_len + MAX_QUESTION_SPACE,
        min_length=origin_seq_len + MIN_QUESTION_SPACE,
        pad_token_id=0,
        bos_token_id=1,
        eos_token_id=2,
        num_beams=5,
        repetition_penalty=1.3,
        no_repeat_ngram_size=3,
        num_return_sequences=1,
    )

    for decoded_tokens in decoded_sequences.tolist():
        decoded_question_text = tokenizer.decode(decoded_tokens[origin_seq_len:])
        decoded_question_text = decoded_question_text.split("</s>")[0].replace("<s>", "")
        generated_results.append(
            (examples[i].context, examples[i].answer, examples[i].question, decoded_question_text)
        )

# with open('test.tsv', "w") as f:
#     for context, answer, question, generated_question in generated_results:
#         f.write(f"{generated_question}\t{answer}\t{context}\n")

### 데이터셋 생성

In [21]:
contexts = []
answers = []
questions = []

for context, answer, question, generated_question in generated_results:
    contexts.append(context)
    answers.append(answer)
    questions.append(generated_question)


In [ ]:
doc_ids = []
context_dict = {}
for i in range(len(wiki_df)):
    context_dict[wiki_df['text'][i]] = wiki_df['document_id'][i]

for context in tqdm(contexts, position=0):
    doc_ids.append(context_dict[context])

In [23]:
answer_start_idx = []

for c, a in zip(contexts, answers):
    answer_start_idx.append(c.find(a))
    
answers_ = [{'answer_start':[i], 'text':[a]} for i, a in zip(answer_start_idx, answers)]

In [ ]:
len(doc_ids), len(generated_results)

In [174]:
train_dataset = load_from_disk("../../data/train_dataset")

In [189]:
qg_train_dict = {
    '__index_level_0__':[0]*len(doc_ids),
    'id':[0]*len(doc_ids),
    'question':questions,
    'title':answers, 
    'context':contexts, 
    'answers':answers_, 
    "document_id":doc_ids,
}


In [190]:
dataset_dict = datasets.DatasetDict({
    "train": datasets.arrow_dataset.Dataset.from_dict(qg_train_dict),
    "validation": train_dataset['validation'],
})

dataset_dict.save_to_disk("../../data/qg_data_v1")

In [198]:
test_dataset = load_from_disk('../../data/qg_data_v1')
test_dataset

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 28765
    })
    validation: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 240
    })
})

In [217]:
pprint(test_dataset['train'][3])

{'__index_level_0__': 0,
 'answers': {'answer_start': [307], 'text': ['대전역 가락국수']},
 'context': '경부선과 호남선의 옛 분기 구조에서 기인하는데, 일제 강점기에 조선총독부에서 호남지역 곡물을 부산으로 이동시켜 수탈해 '
            '가기 쉽게 하기 위하여 호남선의 선구가 경부선의 대전역에서 부산 방향을 향하도록 호남선을 부설하였다.\\n\\n그러다 '
            '보니 서울에서 호남선을 이용하여 호남 지방으로 가려면 현재와는 달리 대전역까지 갔다가 대전역에서 기관차의 방향을 '
            '반대로 바꿔서 가는 것이 불가피하였고, 이 작업이 오랜 시간을 소요하기 때문에 그 시간을 노려서 대전역에서 잠시 내린 '
            '후 승강장의 식당으로 가서 가락국수를 시켜먹는 사람들이 많이 있었다.\\n\\n이 때문에 대전역 가락국수는 자연히 '
            '대전역의 명물이 되었다. 그 후 호남선이 대전역을 경유하지 않고 대전조차장에서 서대전역으로 분기하는 구조로 바뀐 '
            '현재까지도, 승강장에서 도착 열차 대기 중 대전역 가락국수를 찾는 사람들이 있고, 환승열차 30분 대기 중에 '
            '가락국수를 찾는 고객들도 이를 많이 이용하고 있다.\\n\\n가락국수는 대전역뿐만 아니라 경부선에서 장항선이 분기하는 '
            '천안역, 중앙선에서 태백선과 충북선이 분기하는 제천역, 경부선에서 대구선이 분기하는 대구역이나 동대구역 등에서도 '
            '팔았다.',
 'document_id': 20216,
 'id': 0,
 'question': '충청선에서 대전까지 운행한 가락국수의 이름은?',
 'title': '대전역 가락국수'}


In [180]:
pprint(test_dataset['validation'][0])

{'__index_level_0__': 2146,
 'answers': {'answer_start': [284], 'text': ['한보철강']},
 'context': '순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법시험에 합격하여 판사로 임용되었고 '
            '대법원 재판연구관, 수원지법 부장판사, 사법연수원 교수, 특허법원 부장판사 등을 거쳐 능력을 인정받았다. 2003년 '
            '최종영 대법원장의 지명으로 헌법재판소 재판관을 역임하였다.\\n\\n경제민주화위원회(위원장 장하성이 소액주주들을 '
            '대표해 한보철강 부실대출에 책임이 있는 이철수 전 제일은행장 등 임원 4명을 상대로 제기한 손해배상청구소송에서 '
            '서울지방법원 민사합의17부는 1998년 7월 24일에 "한보철강에 부실 대출하여 은행에 막대한 손해를 끼친 점이 '
            '인정된다"며 "원고가 배상을 청구한 400억원 전액을 은행에 배상하라"고 하면서 부실 경영인에 대한 최초의 배상 '
            '판결을 했다. \\n\\n2004년 10월 신행정수도의건설을위한특별조치법 위헌 확인 소송에서 9인의 재판관 중 '
            '유일하게 각하 견해를 내었다. 소수의견에서 전효숙 재판관은 다수견해의 문제점을 지적하면서 관습헌법 법리를 부정하였다. '
            '전효숙 재판관은 서울대학교 근대법학교육 백주년 기념관에서 열린 강연에서, 국회가 고도의 정치적인 사안을 정치로 '
            '풀기보다는 헌법재판소에 무조건 맡겨서 해결하려는 자세는 헌법재판소에게 부담스럽다며 소회를 밝힌 바 있다.',
 'document_id': 9027,
 'id': 'mrc-0-003264',
 'question': '처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?',
 'title': '전효숙'}
